### 1. Initialization and Data Loading

In [ ]:
import pandas as pd
import re
import os
import glob

In [ ]:
# Read one file for analysis. Files must be manually uploaded to /content/submissions folder.
df = pd.read_csv("/content/submissions/RS_2020-07.csv")

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
df_clean = df.copy()

In [ ]:
df_clean.isnull().sum()

I check which submissions are with NaN values in "text" column to decide if they should be deleted:

In [ ]:
mask = df_clean['text'].isnull()
df_clean[mask]

We see that despite the NaN value in "text" column for posts above, there is a title, so there's no need to delete these lines.
Next, check for dublicates.

In [ ]:
duplicate_count = df_clean.duplicated().sum()
duplicate_count

Change case in both title and text to lower and get clear author username.

In [ ]:
df_clean["title"] = df_clean["title"].str.lower()
df_clean["text"] = df_clean["text"].str.lower()

In [ ]:
df_clean["author"] = df_clean["author"].str.strip()
df_clean["author"] = df_clean["author"].str.replace(r"^u/", "", regex=True)

In [ ]:
df_clean

Get rid of all extra symbols and add a column with pure subreddit name:

In [ ]:
df_clean["text"] = (
    df_clean["text"]
    .astype(str)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True) # FIXED: keeps spaces between words
    .str.replace(r"[^a-zA-Z0-9\s'.,!?-]", "", regex=True) # FIXED
)
df_clean["title"] = (
    df_clean["title"]
    .astype(str)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True) # FIXED
    .str.replace(r"[^a-zA-Z0-9\s'.,!?-]", "", regex=True) # FIXED
)

In [ ]:
sub_pattern = r'(?i)(?:https?://)?(?:www\.|old\.|np\.)?reddit\.com/r/([^/?#]+)/?'
df_clean["subreddit"] = df_clean["link"].astype(str).str.extract(sub_pattern)

In [ ]:
df_clean

There are many "[deleted]" values in "author" and "text" columns, so i double-check is there are any lines with both values absent.

In [ ]:
deleted_values = ['deleted', '[deleted]']
mask_text = df_clean['text'].isin(deleted_values)
mask_title = df_clean['title'].isin(deleted_values)
combined_mask = mask_text & mask_title
print(df_clean[combined_mask])

### 2. Batch Processing and Merging

In [ ]:
#function with all the cleaning
def process_reddit_file(df):
    df_clean = df.copy()

    df_clean["author"] = df_clean["author"].str.strip().str.replace(r"^u/", "", regex=True)

    df_clean["text"] = (
        df_clean["text"]
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.replace(r"[^a-zA-Z0-9\s'.,!?-]", "", regex=True)
    )

    df_clean["title"] = (
        df_clean["title"]
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.replace(r"[^a-zA-Z0-9\s'.,!?-]", "", regex=True)
    )

    sub_pattern = r'(?i)(?:https?://)?(?:www\.|old\.|np\.)?reddit\.com/r/([^/?#]+)/?'
    df_clean["subreddit"] = df_clean["link"].astype(str).str.extract(sub_pattern)


    empty_values = ['deleted', '[deleted]', 'nan']
    mask_text_empty = df_clean['text'].isin(empty_values)
    mask_title_empty = df_clean['title'].isin(empty_values)
    empty_both_mask = mask_text_empty & mask_title_empty
    
    duplicates_mask = df_clean.duplicated(subset=["author", "title", "text", "created"])

    final_delete_mask = duplicates_mask | empty_both_mask
    
    #drop unnecessary column (as requested)
    if 'url' in df_clean.columns:
        df_clean = df_clean.drop(columns=['url'])

    df_processed = df_clean[~final_delete_mask]
    
    return df_processed


In [ ]:
# Setup local paths (no Google Drive required)
base_dir = "/content"
submissions_dir = os.path.join(base_dir, "submissions")
output_dir = submissions_dir # Save in the same directory

# Create the submissions directory if it doesn't exist (needed for batch process to run)
os.makedirs(submissions_dir, exist_ok=True)

file_pattern = os.path.join(submissions_dir, "RS_*.csv")
csv_files = glob.glob(file_pattern)

print(f"Found {len(csv_files)} files to process.")
print("--- Starting batch processing ---")

for file_path in csv_files:
    filename = os.path.basename(file_path)
    print(f"--- Processing: {filename} ---")
    
    try:
        # Read
        df = pd.read_csv(file_path)
        initial_rows = len(df)
        
        # Process using our function
        df_clean = process_reddit_file(df)
        final_rows = len(df_clean)
        
        # Save
        output_filename = f"cleaned_{filename}"
        output_path = os.path.join(output_dir, output_filename)
        df_clean.to_csv(output_path, index=False)
        
        print(f"  Initial: {initial_rows}, Final: {final_rows}, Removed: {initial_rows - final_rows}")
        print(f"  Saved to: {output_path}\n")
    
    except Exception as e:
        print(f"*** ERROR processing {filename}: {e} ***\n")

print("--- All processing complete. ---")

And finally merge all cleaned files together.

In [ ]:
print("--- Starting merge process ---")

# 1. Find all cleaned files 
cleaned_pattern = os.path.join(output_dir, "cleaned_RS_*.csv")
cleaned_files = glob.glob(cleaned_pattern)

print(f"Found {len(cleaned_files)} cleaned files to merge.")

# 2. Use a generator to read files and concatenate them
df_generator = (pd.read_csv(f) for f in cleaned_files)
merged_df = pd.concat(df_generator, ignore_index=True)

# 3. Define save path (saving in the universal /content directory)
merged_filename = "merged_submissions.csv"
merged_path = os.path.join(base_dir, merged_filename)

# 4. Save the final merged file
merged_df.to_csv(merged_path, index=False)

print("--- Merge complete! ---")
print(f"Total rows in merged file: {len(merged_df)}")

In [ ]:
# Calculate and print the merged file size
mem_usage_mb = merged_df.memory_usage(deep=True).sum() / (1024**2)
print(f"Total memory size: {mem_usage_mb:.2f} MB")

In [ ]:
# Read the final merged file saved to the universal /content folder
df_merged = pd.read_csv("/content/merged_submissions.csv")

In [ ]:
df_merged.head()

In [ ]:
# Optional: View overall stats as a DataFrame
row_count = len(df_merged)
mem_usage_mb = df_merged.memory_usage(deep=True).sum() / (1024**2)

stats_data = {
    'RowCount': [row_count],
    'Size_MB': [mem_usage_mb]
}
stats_df = pd.DataFrame(stats_data)
stats_df['Size_MB'] = stats_df['Size_MB'].round(4)
print("DataFrame with overall statistics:")
stats_df

In [ ]:
# Optional: Frequency overview of data types
print("--- Frequency overview of data types (merged_df) ---")
data_type_frequency = df_merged.dtypes.value_counts()
print(data_type_frequency)